문제 정의 : SVM(support vector machine) 사용하여 타이타닉 생존자(1), 사망(0) 예측

기본 라이브러리 불러오기


In [13]:
import pandas as pd
import seaborn as sns

한글깨짐 방지

In [14]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


[step1] 데이터 준비- Seaborn에서 제공하는 titanic 데이터셋 가져오기

In [15]:
#load_dataset 함수를 사용하여 데이터프레임으로 변환
df=sns.load_dataset('titanic')

[step2] 데이터 탐색 및 데이터 전처리


In [16]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [17]:
df.info()
#deck에 null이 너무 많아서 삭제했음.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [18]:
#NaN값이 많은 deck(객실 데크 위치) 열을 삭제, embarked와 내용이 겹치는 embark_town(승선도시) 열을 삭제
rdf=df.drop(['deck','embark_town'], axis=1)

In [24]:
# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값 확인 -> 탑승한 도시의 첫글자
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
most_freq


'S'

In [27]:
# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
rdf['embarked'].fillna(most_freq, inplace=True)

In [25]:
# age 열에 나이 데이터가 없는 모든 행을 삭제-> 177개 NaN삭제
rdf=rdf.dropna(subset=['age'], how='any', axis=0)
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    714 non-null    int64   
 1   pclass      714 non-null    int64   
 2   sex         714 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       714 non-null    int64   
 5   parch       714 non-null    int64   
 6   fare        714 non-null    float64 
 7   embarked    712 non-null    object  
 8   class       714 non-null    category
 9   who         714 non-null    object  
 10  adult_male  714 non-null    bool    
 11  alive       714 non-null    object  
 12  alone       714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


In [28]:
# 분석에 활용할 열(속성)을 선택(생존여부, 객실등급, 성별, 나이, 타이타닉에 탑승한 형제/자매수,
#부보자녀수, 탑승한 곳 (항구) C=cherbourg, W=Queenstown, s=Southampton)

ndf=rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [29]:
# 원핫인코딩-범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변경
#male, female->[1,0],[0,1]
#embarked C,Q,S->[1,0,0],[0,1,0],[0,0,1]

onehot_sex=pd.get_dummies(ndf['sex'])
onehot_sex
ndf=pd.concat([ndf, onehot_sex], axis=1)
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male
0,0,3,male,22.0,1,0,S,0,1
1,1,1,female,38.0,1,0,C,1,0
2,1,3,female,26.0,0,0,S,1,0
3,1,1,female,35.0,1,0,S,1,0
4,0,3,male,35.0,0,0,S,0,1


[step4] 데이터셋 구분- 훈련용(train data)/ 검증용(test data)

In [30]:
#embarked C,Q,S->[1,0,0],[0,1,0],[0,0,1]

onehot_embarked=pd.get_dummies(ndf['embarked'], prefix='town')
onehot_embarked
ndf=pd.concat([ndf, onehot_embarked], axis=1)
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1


In [31]:
# 기존 컬럼 삭제
ndf.drop(['sex','embarked'], axis=1, inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [59]:
# 속성(변수)선택
X=ndf[['pclass','age','sibsp','parch','female',	'male',	
       'town_C',	'town_Q',	'town_S']] #독립변수
y=ndf['survived']

In [60]:
# 설명 변수(데이터, 문제집) 정규화(normalization) -> 스케일링 0~1사이의 작은 값으로 축소

from sklearn import  preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

array([[ 0.91123237, -0.53037664,  0.52457013, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364,  0.57183099,  0.52457013, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237, -0.25482473, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       ...,
       [-1.47636364, -0.73704057, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364, -0.25482473, -0.55170307, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237,  0.15850313, -0.55170307, ..., -0.47180795,
         4.94974747, -1.87589641]])

In [61]:
#train 데이터와 test 데이터로 구분(7:3 비율)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=7)

print(X_train.shape)
print(X_test.shape)

(499, 9)
(215, 9)


[step5] SVM분류 모형 -sklearn 사용

In [64]:
#sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import  svm

# 모형 객체 생성 (kernel='rbf' 적용)
# 벡터 공간을 맵핑 하는 함수 -> radial basis function (RBF) 커널을 사용
# 선형(linear), 다항식(poly), 가우시안 RBF(rbf) , 시그모이드(sigmoid)
svm_model = svm.SVC(kernel='rbf')# 벡터 공간을 맵핑 하는 함수 -> 기본 rbf 사용


In [65]:
# 훈련 데이터를 사용하여 모형 학습
svm_model.fit(X_train, y_train)

# 테스트 데이터를 사용하여 y_hat 예측
y_hat=svm_model.predict(X_test)

[step6] 모형 성능 평가

In [66]:
from sklearn import  metrics

print('테스트 세트 정확도 : {:.2f}%'.format(svm_model.score(X_test,y_test)*100))

테스트 세트 정확도 : 80.47%


In [58]:
#모형 성능 평가 - 평가지표 계산
svm_report=metrics.classification_report(y_test, y_hat)
print(svm_report)

              precision    recall  f1-score   support

           0       0.61      0.95      0.74       126
           1       0.67      0.13      0.22        89

    accuracy                           0.61       215
   macro avg       0.64      0.54      0.48       215
weighted avg       0.63      0.61      0.53       215

